In [63]:
from datacube.api import GridWorkflow
import matplotlib
from madmex.wrappers import gwf_query
from madmex.util import join_dicts
import xarray as xr
from madmex.util import yaml_to_dict, mid_date, parser_extra_args
from datetime import datetime
import os
from datacube.storage import masking
from madmex.util.xarray import to_float, to_int

In [11]:
product = ['ls5_espa_mexico','ls7_espa_mexico_50km']
gwf =  {'region': 'Oaxaca', 'begin': '2004-01-01', 'end': '2005-12-31'}

In [13]:
dict_list = []
for prod in product:
            gwf_kwargs.update(product = prod)
            try:
                dict_list.append(gwf_query(**gwf_kwargs, view=False))
            # Exception is in case one of the product hasn't been registered in the datacube
            except Exception as e:
                pass
iterable = join_dicts(*dict_list, join='full').items()

In [20]:
lista_iter = list(iterable)
[lista_iter[x][0] for x in range(0,len(lista_iter))]

[(40, -40),
 (42, -34),
 (38, -35),
 (45, -39),
 (39, -40),
 (41, -36),
 (44, -40),
 (46, -37),
 (37, -39),
 (38, -39),
 (40, -35),
 (43, -37),
 (42, -37),
 (39, -36),
 (41, -40),
 (44, -36),
 (40, -39),
 (42, -38),
 (47, -39),
 (39, -37),
 (44, -39),
 (42, -40),
 (43, -38),
 (42, -41),
 (38, -38),
 (40, -38),
 (42, -36),
 (41, -37),
 (39, -38),
 (44, -38),
 (46, -39),
 (42, -39),
 (41, -41),
 (43, -35),
 (41, -34),
 (38, -37),
 (40, -37),
 (43, -39),
 (42, -35),
 (41, -38),
 (47, -37),
 (44, -37),
 (38, -36),
 (43, -36),
 (45, -38),
 (39, -39),
 (41, -35),
 (46, -38),
 (38, -40),
 (40, -36),
 (43, -40),
 (39, -35),
 (41, -39),
 (47, -38)]

In [21]:
var = lista_iter[0]

In [24]:
var[1][0].geobox

GeoBox(1667, 1667, Affine(30.0, 0.0, 2977560.0,
       0.0, -30.0, 476330.0), PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]])

In [27]:
sr_0 = xr.merge([GridWorkflow.load(x, dask_chunks={'x': 1667, 'y': 1667}) for x in var[1]])

In [28]:
sr_0

<xarray.Dataset>
Dimensions:   (time: 24, x: 1667, y: 1667)
Coordinates:
  * time      (time) datetime64[ns] 1999-01-08T16:27:29 ... 2004-12-31T16:37:47
  * y         (y) float64 4.763e+05 4.763e+05 4.763e+05 ... 4.264e+05 4.263e+05
  * x         (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Data variables:
    blue      (time, y, x) float64 dask.array<shape=(24, 1667, 1667), chunksize=(24, 1667, 1667)>
    green     (time, y, x) float64 dask.array<shape=(24, 1667, 1667), chunksize=(24, 1667, 1667)>
    red       (time, y, x) float64 dask.array<shape=(24, 1667, 1667), chunksize=(24, 1667, 1667)>
    nir       (time, y, x) float64 dask.array<shape=(24, 1667, 1667), chunksize=(24, 1667, 1667)>
    swir1     (time, y, x) float64 dask.array<shape=(24, 1667, 1667), chunksize=(24, 1667, 1667)>
    swir2     (time, y, x) float64 dask.array<shape=(24, 1667, 1667), chunksize=(24, 1667, 1667)>
    pixel_qa  (time, y, x) float64 dask.array<shape=(24, 1667, 1667), chunksize=(24

In [49]:
crs = var[1][0].geobox.crs
begin = datetime.strptime('2004-01-01', '%Y-%m-%d')
end = datetime.strptime('2005-12-31', '%Y-%m-%d')
center_dt = mid_date(begin, end)
center_dt = center_dt.strftime("%Y-%m-%d")
path = "/shared_volume/datacube/datacube_ingest/recipes/l57_oax_recipe_9905_1"
#os.makedirs(path)
nc_filename = os.path.join(path, 'madmex_002_%d_%d_%s.nc' % (var[0][0], var[0][1], center_dt))

In [57]:
sr_0_combine = xr.auto_combine([GridWorkflow.load(x, dask_chunks={'x': 1667, 'y': 1667}) for x in var[1]], concat_dim='time')

In [58]:
sr_0_combine

<xarray.Dataset>
Dimensions:   (time: 24, x: 1667, y: 1667)
Coordinates:
  * y         (y) float64 4.763e+05 4.763e+05 4.763e+05 ... 4.264e+05 4.263e+05
  * x         (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
  * time      (time) datetime64[ns] 1999-01-08T16:27:29 ... 2004-12-31T16:37:47
Data variables:
    blue      (time, y, x) int16 dask.array<shape=(24, 1667, 1667), chunksize=(16, 1667, 1667)>
    green     (time, y, x) int16 dask.array<shape=(24, 1667, 1667), chunksize=(16, 1667, 1667)>
    red       (time, y, x) int16 dask.array<shape=(24, 1667, 1667), chunksize=(16, 1667, 1667)>
    nir       (time, y, x) int16 dask.array<shape=(24, 1667, 1667), chunksize=(16, 1667, 1667)>
    swir1     (time, y, x) int16 dask.array<shape=(24, 1667, 1667), chunksize=(16, 1667, 1667)>
    swir2     (time, y, x) int16 dask.array<shape=(24, 1667, 1667), chunksize=(16, 1667, 1667)>
    pixel_qa  (time, y, x) uint16 dask.array<shape=(24, 1667, 1667), chunksize=(16, 1667, 1667)

In [59]:
sr_0_combine.attrs['geobox'] = var[1][0].geobox

In [60]:
clear = masking.make_mask(sr_0_combine.pixel_qa, cloud=False, cloud_shadow=False, snow=False)

In [64]:
sr_1 = sr_0_combine.where(clear)
sr_1 = sr_0_combine.drop('pixel_qa')
sr_1 = sr_0_combine.apply(func=to_float, keep_attrs=True)

In [66]:
sr_1['ndvi'] = ((sr_1.nir - sr_1.red) / (sr_1.nir + sr_1.red)) * 10000
sr_1['ndvi'].attrs['nodata'] = -9999

In [68]:
sr_1['ndmi'] = ((sr_1.nir - sr_1.swir1) / (sr_1.nir + sr_1.swir1)) * 10000
sr_1['ndmi'].attrs['nodata'] = -9999